<a href="https://colab.research.google.com/github/thiagodepaulo/nlp/blob/enap/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin

In [6]:
## carrega base de dados
categories = ['alt.atheism', 'talk.religion.misc',
               'comp.graphics', 'sci.space']

newsgroups = fetch_20newsgroups(subset='all', remove=('headers','footers','quotes'), categories=categories)

In [47]:
## cria pipeline
pipeline = Pipeline([
    ('sw', StopWord()),
    ('tfidf', TfidfVectorizer()),
    ('clf',MultinomialNB())
    ])

param = {
    'tfidf__max_df':[0.9, 0.95],
    'clf__alpha':[1.0, 0.01,]
}

scoring = {
        'accuracy',
        'f1_micro',
        'f1_macro',
        'f1_weighted'
    }

grid_search = GridSearchCV(pipeline, param, n_jobs=-1, verbose=1,
                           return_train_score=False, scoring=scoring, refit='f1_macro')

grid_search.fit(newsgroups.data, newsgroups.target)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


GridSearchCV(estimator=Pipeline(steps=[('sw', StopWord()),
                                       ('tfidf', TfidfVectorizer()),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [1.0, 0.01],
                         'sw__lang': [None, 'english'],
                         'tfidf__max_df': [0.9, 0.95]},
             refit='f1_macro',
             scoring={'f1_weighted', 'f1_macro', 'accuracy', 'f1_micro'},
             verbose=1)

In [48]:
pd.DataFrame(grid_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__alpha,param_sw__lang,param_tfidf__max_df,params,split0_test_f1_weighted,split1_test_f1_weighted,...,std_test_accuracy,rank_test_accuracy,split0_test_f1_micro,split1_test_f1_micro,split2_test_f1_micro,split3_test_f1_micro,split4_test_f1_micro,mean_test_f1_micro,std_test_f1_micro,rank_test_f1_micro
0,7.886786,0.282667,2.003140,0.295970,1.0,None,0.9,"{'clf__alpha': 1.0, 'sw__lang': None, 'tfidf__...",0.753877,0.761736,...,0.008506,5,0.780236,0.783186,0.774003,0.765140,0.790251,0.778563,0.008506,5
1,7.952367,0.148602,1.825375,0.138995,1.0,None,0.95,"{'clf__alpha': 1.0, 'sw__lang': None, 'tfidf__...",0.753877,0.761736,...,0.008506,5,0.780236,0.783186,0.774003,0.765140,0.790251,0.778563,0.008506,5
2,7.434527,0.176681,2.056889,0.372074,1.0,english,0.9,"{'clf__alpha': 1.0, 'sw__lang': 'english', 'tf...",0.753877,0.761736,...,0.008506,5,0.780236,0.783186,0.774003,0.765140,0.790251,0.778563,0.008506,5
3,7.952620,0.115250,1.822435,0.136624,1.0,english,0.95,"{'clf__alpha': 1.0, 'sw__lang': 'english', 'tf...",0.753877,0.761736,...,0.008506,5,0.780236,0.783186,0.774003,0.765140,0.790251,0.778563,0.008506,5
4,7.555252,0.297104,2.075577,0.247127,0.01,None,0.9,"{'clf__alpha': 0.01, 'sw__lang': None, 'tfidf_...",0.845335,0.858186,...,0.009529,1,0.846608,0.859882,0.843427,0.847858,0.830133,0.845582,0.009529,1
5,7.820908,0.373142,1.825376,0.138939,0.01,None,0.95,"{'clf__alpha': 0.01, 'sw__lang': None, 'tfidf_...",0.845335,0.858186,...,0.009529,1,0.846608,0.859882,0.843427,0.847858,0.830133,0.845582,0.009529,1
6,7.744524,0.374874,1.903249,0.177362,0.01,english,0.9,"{'clf__alpha': 0.01, 'sw__lang': 'english', 't...",0.845335,0.858186,...,0.009529,1,0.846608,0.859882,0.843427,0.847858,0.830133,0.845582,0.009529,1
7,7.680345,0.338281,1.876804,0.391296,0.01,english,0.95,"{'clf__alpha': 0.01, 'sw__lang': 'english', 't...",0.845335,0.858186,...,0.009529,1,0.846608,0.859882,0.843427,0.847858,0.830133,0.845582,0.009529,1


In [39]:
import nltk

## stop word
class StopWord(BaseEstimator, TransformerMixin):

  def __init__(self, lang='english'):
    self.lang = lang
    self.stop_list = nltk.corpus.stopwords.words(lang)

  def fit(self, X, y=None):
    return self

  def remove_tokens(self, doc):
    doc_tkn = nltk.word_tokenize(doc)
    return [token for token in doc_tkn if token.lower() not in self.stop_list]

  def transform(self, X):
    return [' '.join(self.remove_tokens(doc)) for doc in X]

In [29]:
sw = StopWord()

X_sw = sw.transform(newsgroups.data)
print(X_sw[0])


point set views way believe . Saying eveil world caused atheism ridiculous counterproductive dialogue newsgroups . see posts spirit condemnation atheists newsgroup bacause don' believe exactly . 're try convert atheists , 're failing miserably . wants position constantly defending agaist insulting attacks , like seem like ? ! 'm sorry 're blind n't get messgae quote , everyone else seemed .


In [23]:
print

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True